<a href="https://colab.research.google.com/github/TomographicImaging/gVXR-Tutorials/blob/main/notebooks/segmentation-to-CT_scan-simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# -*- coding: utf-8 -*-
#
#  Copyright 2025 United Kingdom Research and Innovation
#
#  Licensed under the Apache License, Version 2.0 (the "License");
#  you may not use this file except in compliance with the License.
#  You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
#  Unless required by applicable law or agreed to in writing, software
#  distributed under the License is distributed on an "AS IS" BASIS,
#  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#  See the License for the specific language governing permissions and
#  limitations under the License.
#
#   Authored by:    Franck Vidal (UKRI-STFC)

![gVXR](https://github.com/TomographicImaging/gVXR-Tutorials/blob/main/img/Logo-transparent-small.png?raw=1)

# Digital twin of a CT synchrotron beamline: [Dual Imaging And Diffraction (DIAD)](https://www.diamond.ac.uk/Instruments/Imaging-and-Microscopy/DIAD.html)

Summary here.
<!-- This notebook shows how to load a segmented image and you it to create a multi-part sample. 
It relies on functions built in gVXR, i.e. no third-party software is required. 
The functionality is multi-threaded to boost performances. 
It is then used to simulate a CT scan acquisition with gVXR and recontruct it with CIL. -->

![Illustration here]()

<div class="alert alert-block alert-warning">
    <b>Note:</b> Make sure the Python packages are already installed. See <a href="../README.md">README.md</a> in the root directory of the repository. If you are running this notebook from Google Colab, please run the cell below to install the package with `!pip install gvxr`
</div>

In [2]:
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    !apt-get install libnvidia-gl-575
    !pip install -q condacolab
    import condacolab
    condacolab.install()

    !conda install -y -c conda-forge -c https://software.repos.intel.com/python/conda -c ccpi cil=24.3.0 ipp=2021.12 tigre

    !pip install gvxr

In [3]:
import os

# import os # Create the output directory if necessary
# import numpy as np # Who does not use Numpy?

# from tifffile import  imread
# import matplotlib.pyplot as plt # Plotting
import matplotlib.pyplot as plt

# from tqdm.notebook import tqdm

# User interface
import IPython
from IPython.display import display
from ipywidgets import widgets, Layout

#  CT simulation

from gvxrPython3 import gvxr
# from gvxrPython3.gVXRDataReader import *

# # CT reconstruction
# from cil.recon import FDK # For CBCT
# from cil.recon import FBP # For parallel beam geometry

# from cil.io import TIFFWriter

# from cil.processors import TransmissionAbsorptionConverter
# from cil.utilities.display import show_geometry, show2D

Create a simulation environment

In [4]:
# Create an OpenGL context
# MS Windows
if os.name == "nt":
    print("Creating an OpenGL context using EGL is not possible on Microsoft Windows")
    print("A window will be created but not displayed")
    gvxr.createWindow(0, False, "OPENGL")
# MacOS
elif str(os.uname()).find("Darwin") >= 0:
    print("Creating an OpenGL context using EGL is not possible on MacOS")
    print("A window will be created but not displayed")
    gvxr.createWindow(0, False, "OPENGL")
# GNU/Linux
else:
    print("Create an OpenGL context using EGL")
    gvxr.createWindow(0, 1, "EGL")


Create an OpenGL context using EGL


Tue Aug  5 16:57:11 2025 ---- Create window (ID: 0)
Tue Aug  5 16:57:11 2025 ---- Request a EGL context
Tue Aug  5 16:57:11 2025 ---- EGL client extensions:
Tue Aug  5 16:57:11 2025 ---- EGL_EXT_platform_base EGL_EXT_device_base EGL_EXT_device_enumeration EGL_EXT_device_query EGL_KHR_client_get_all_proc_addresses EGL_EXT_client_extensions EGL_KHR_debug EGL_KHR_platform_x11 EGL_EXT_platform_x11 EGL_EXT_platform_device EGL_MESA_platform_surfaceless EGL_EXT_explicit_device EGL_KHR_platform_wayland EGL_EXT_platform_wayland EGL_KHR_platform_gbm EGL_MESA_platform_gbm EGL_EXT_platform_xcb
Tue Aug  5 16:57:11 2025 ---- EGL clients supported by this host:
Tue Aug  5 16:57:11 2025 ---- 	- Surfaceless platform
Tue Aug  5 16:57:11 2025 ---- 	- X11 platform
Tue Aug  5 16:57:11 2025 ---- 	- Wayland platform
Tue Aug  5 16:57:11 2025 ---- 	- GBM platform
Tue Aug  5 16:57:11 2025 ---- 	- Device platform
Tue Aug  5 16:57:11 2025 ---- Initialise EGL
Tue Aug  5 16:57:11 2025 ---- Try to build an EGL displ

Import the twin

In [5]:
from gvxrPython3.twins.utils import createDigitalTwin

In [6]:
twin = createDigitalTwin(name="DIAD")

In [7]:
# twin.get_beams()

In [8]:
# twin.source

In [9]:
# twin.set_beam(twin.get_beams()[1])
# print(twin.source)
# print(twin.beam)

Class to create a small UI

In [10]:
def get_available_filters(twin, beam_type):
    return twin.specification.beams[beam_type].filters




In [11]:

class UI:
    def __init__(self):
        pass

    def get_filter_material(self, twin, beam_type):

        filter_materials = ["None"]

        for item in get_available_filters(twin, beam_type):
            filter_materials.append(item[0])

        return filter_materials

    def get_filter_thickness(self, twin, beam_type, material):

        thicknesses = [str(0.0)]

        filters = twin.specification.beams[beam_type].filters

        for item in filters:

            if item[0] == material:
                for thickness in item[1:]:
                    thicknesses.append(str(thickness))

        return thicknesses

    def create(self):

        global twin


        # Select the spectrum to use
        # spectrum_type_widget = widgets.RadioButtons(
        #     options=['Monochromatic', 'Pink-beam (low angle)', 'Pink-beam (high angle)'],
        #     # description='Beam spectrum:',
        #     disabled=False,
        #     value='Monochromatic',
        #     layout={'width': 'max-content'}
        # )

        self.spectrum_type_widget = widgets.Select(
            options=twin.get_beams(),
            # description='Beam spectrum:',
            disabled=False,
            value=twin.get_beams()[0],
            layout={'width': 'max-content'}
        )

        # Make it look nice
        self.beam_selection_box = widgets.Box(
            [
                widgets.Label(value='Select the type of spectrum:'), 
                self.spectrum_type_widget
            ]
        )

        # If you selected a pink beam, i.e. a poychromatic beam as the monochromator was not used, you may select a filter.
        self.filter_material_type_widget = widgets.Select(
            options=self.get_filter_material(twin, self.spectrum_type_widget.value),
            # description='Filter material:',
            disabled=False,
            value='None',
            layout={'width': 'max-content'}
        )

        # If you selected a pink beam, i.e. a poychromatic beam as the monochromator was not used, you may select a filter.
        filter_material = self.filter_material_type_widget.value
        thicknesses = self.get_filter_thickness(twin, self.spectrum_type_widget.value, filter_material)
        self.filter_thickness_widget = widgets.Select(
            options=thicknesses,
            # description='Filter thickness [mm]:',
            disabled=False,
            value=thicknesses[0],
            layout={'width': 'max-content'}
        )

        self.filter_selection_box = widgets.Box(
            [
                widgets.Label(value='Filter material:'), 
                self.filter_material_type_widget, 
                widgets.Label(value='Filter thickness [mm]:'), 
                self.filter_thickness_widget
            ]
        )

        # Else select the energy
        twin_max_energy = max(twin.specification.beams["monochromatic"].keV)
        twin_min_energy = min(twin.specification.beams["monochromatic"].keV)
        self.energy_selector_widget = widgets.FloatSlider(
            value=twin_min_energy + (twin_max_energy - twin_min_energy) / 2.0,
            min=twin_min_energy,
            max=twin_max_energy,
            step=0.1,
            # description='Mono-energy [keV]:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='.1f',
        )

        # Make it look nice
        self.mono_energy_box = widgets.Box(
            [
                widgets.Label(value='Mono-energy [keV]:'), 
                self.energy_selector_widget
            ]
        )


        if self.spectrum_type_widget.value == 'Monochromatic':
            self.filter_selection_box.disabled = True

            for child in self.filter_selection_box.children:
                child.disabled = True

            for child in self.mono_energy_box.children:
                child.disabled = False

        else:
            self.filter_selection_box.disabled = False

            for child in self.filter_selection_box.children:
                child.disabled = False

            for child in self.mono_energy_box.children:
                child.disabled = True
        
        # Select the exposure
        self.exposure_widget = widgets.FloatSlider(
            value=0.1,
            min=0.5,
            max=1.0,
            step=0.1,
            # description='Exposure [sec]:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='.1f',
        )

        # Make it look nice
        self.exposure_box = widgets.Box(
            [
                widgets.Label(value='Exposure [sec]:'), 
                self.exposure_widget
            ]
        )

        # Plot the correspoding spectrum



        self.spectrum_plot_widget = widgets.Output(layout=Layout(width='50%'))


        def on_spectrum_type_value_change(change):

            twin.set_beam(change['new'])

            # Update the UI
            if change['new'] == 'Monochromatic':
                self.filter_selection_box.disabled = True

                for child in self.filter_selection_box.children:
                    child.disabled = True

                for child in self.mono_energy_box.children:
                    child.disabled = False

            else:
                self.filter_selection_box.disabled = False

                for child in self.filter_selection_box.children:
                    child.disabled = False

                for child in self.mono_energy_box.children:
                    child.disabled = True

            # Update the values in the UI
            self.filter_material_type_widget.value = "None"
            self.filter_material_type_widget.options = self.get_filter_material(twin, self.spectrum_type_widget.value)

            filter_material = self.filter_material_type_widget.value
            thicknesses = self.get_filter_thickness(twin, self.spectrum_type_widget.value, filter_material)
            self.filter_thickness_widget.value = thicknesses[0]
            self.filter_thickness_widget.options = thicknesses

            # Plot the spectrum
            self.update_spectrum_plot()

        def on_filter_material_value_change(change):

            # Update the values in the UI
            filter_material = self.filter_material_type_widget.value
            thicknesses = self.get_filter_thickness(twin, self.spectrum_type_widget.value, filter_material)
            self.filter_thickness_widget.value = thicknesses[0]
            self.filter_thickness_widget.options = thicknesses


            twin.beam.filter = [self.filter_material_type_widget.value,
                float(self.filter_thickness_widget.value),
            ]

            # Plot the spectrum
            self.update_spectrum_plot()


        def on_filter_thickness_value_change(change):

            twin.beam.filter = [self.filter_material_type_widget.value,
                float(self.filter_thickness_widget.value),
            ]

            # Plot the spectrum
            self.update_spectrum_plot()

        def on_energy_value_change(change):

            global twin

            # Change the energy
            twin.beam.kev = change["new"]

            self.update_spectrum_plot()

        def on_exposure_value_change(change):

            global twin

            # Change the energy
            twin.detector.exposure = change["new"]

            self.update_spectrum_plot()


        # Update the visibility of widgets
        self.spectrum_type_widget.observe(on_spectrum_type_value_change, names='value')

        # Update the filter thicknesses based on the material
        self.filter_material_type_widget.observe(on_filter_material_value_change, names='value')
        self.filter_thickness_widget.observe(on_filter_thickness_value_change, names='value')

        # Update the spectrum
        self.energy_selector_widget.observe(on_energy_value_change, names='value')
        self.exposure_widget.observe(on_exposure_value_change, names='value')



        # self.spectrum_type_widget.observe(on_energy_value_change, names='value')
        # self.filter_material_type_widget.observe(on_energy_value_change, names='value')
        # self.filter_thickness_widget.observe(on_energy_value_change, names='value')


        self.layout_left_column = widgets.VBox([self.beam_selection_box, self.mono_energy_box, self.filter_selection_box, self.exposure_box])
        self.layout_two_columns = widgets.HBox([self.layout_left_column, self.spectrum_plot_widget])


        display(self.layout_two_columns)

    def update_spectrum_plot(self):

        twin.apply()

        energy_bins = gvxr.getEnergyBins("keV")
        photon_counts = gvxr.getPhotonCountsPerPixelAtSDD()

        print(energy_bins)
        print(photon_counts)

        with self.spectrum_plot_widget:
            # print(change["new"])


            
            IPython.display.clear_output(True)
            fig = plt.figure(figsize=(10,5), dpi=90)
            fig.canvas.toolbar_visible = False
            ax = fig.add_subplot(111)
            ax.plot(energy_bins, photon_counts)
            ax.set_xlabel("Incident energy [keV]")
            ax.set_ylabel("Number of photons per pixels")
            plt.show()

diad_ui = UI()
diad_ui.create()

Beam filter validation is not implemented
(1.5436417888849974, 2.447165083140135, 3.4279278479516506, 4.408909007906914, 5.38988970220089, 6.370870862156153, 7.3518515564501286, 8.332832716405392, 9.313813410699368, 10.294800624251366, 11.275775730609894, 12.256750836968422, 13.237746432423592, 14.218712225556374, 15.199700370430946, 16.18068851530552, 17.1616543084383, 18.14264990389347, 19.12362314760685, 20.104600116610527, 21.085593849420547, 22.06655777990818, 23.04754965007305, 24.028532207012177, 25.009501725435257, 25.990497320890427, 26.971468701958656, 27.952449396252632, 28.933441266417503, 29.914407059550285, 30.895398929715157, 31.876377761363983, 32.85735100507736, 33.83834660053253, 34.819312393665314, 35.80030053853989, 36.78128868341446, 37.76225447654724, 38.74325007200241, 39.72422331571579, 40.70520028471947, 41.68619588017464, 42.66716167330742, 43.64814981818199, 44.62913051247597, 45.610103756189346, 46.591099351644516, 47.572068870067596, 48.55283349752426, 49.4

Tue Aug  5 16:57:17 2025 (WW) addEnergyBinToSpectrum is deprecated. It will be removed in a future release. Choose addEnergyBinToSpectrumPerPixelAtSDD or addEnergyBinToSpectrumPerCm2At1m. Note that addEnergyBinToSpectrum and addEnergyBinToSpectrumPerPixelAtSDD are equivalent.


Beam filter validation is not implemented
(1.5436417888849974, 2.447165083140135, 3.4279278479516506, 4.408909007906914, 5.38988970220089, 6.370870862156153, 7.3518515564501286, 8.332832716405392, 9.313813410699368, 10.294800624251366, 11.275775730609894, 12.256750836968422, 13.237746432423592, 14.218712225556374, 15.199700370430946, 16.18068851530552, 17.1616543084383, 18.14264990389347, 19.12362314760685, 20.104600116610527, 21.085593849420547, 22.06655777990818, 23.04754965007305, 24.028532207012177, 25.009501725435257, 25.990497320890427, 26.971468701958656, 27.952449396252632, 28.933441266417503, 29.914407059550285, 30.895398929715157, 31.876377761363983, 32.85735100507736, 33.83834660053253, 34.819312393665314, 35.80030053853989, 36.78128868341446, 37.76225447654724, 38.74325007200241, 39.72422331571579, 40.70520028471947, 41.68619588017464, 42.66716167330742, 43.64814981818199, 44.62913051247597, 45.610103756189346, 46.591099351644516, 47.572068870067596, 48.55283349752426, 49.4

Beam filter validation is not implemented
(1.5436417888849974, 2.447165083140135, 3.4279278479516506, 4.408909007906914, 5.38988970220089, 6.370870862156153, 7.3518515564501286, 8.332832716405392, 9.313813410699368, 10.294800624251366, 11.275775730609894, 12.256750836968422, 13.237746432423592, 14.218712225556374, 15.199700370430946, 16.18068851530552, 17.1616543084383, 18.14264990389347, 19.12362314760685, 20.104600116610527, 21.085593849420547, 22.06655777990818, 23.04754965007305, 24.028532207012177, 25.009501725435257, 25.990497320890427, 26.971468701958656, 27.952449396252632, 28.933441266417503, 29.914407059550285, 30.895398929715157, 31.876377761363983, 32.85735100507736, 33.83834660053253, 34.819312393665314, 35.80030053853989, 36.78128868341446, 37.76225447654724, 38.74325007200241, 39.72422331571579, 40.70520028471947, 41.68619588017464, 42.66716167330742, 43.64814981818199, 44.62913051247597, 45.610103756189346, 46.591099351644516, 47.572068870067596, 48.55283349752426, 49.4

Beam filter validation is not implemented
(32.85735100507736, 33.83834660053253, 34.819312393665314, 35.80030053853989, 36.78128868341446, 37.76225447654724, 38.74325007200241, 39.72422331571579, 40.70520028471947, 41.68619588017464, 42.66716167330742, 43.64814981818199, 44.62913051247597, 45.610103756189346, 46.591099351644516, 47.572068870067596, 48.55283349752426, 49.45637285709381)
(0.0003338598762638867, 0.00622920785099268, 0.08470340818166733, 0.8742391467094421, 7.090804576873779, 46.505096435546875, 253.03884887695312, 1167.06103515625, 4644.033203125, 16230.4697265625, 50486.0078125, 141519.421875, 361319.46875, 848706.3125, 1847409.75, 3757473.0, 7187282.0, 12420765.0)


...

Beam filter validation is not implemented
(10.294800624251366, 11.275775730609894, 12.256750836968422, 13.237746432423592, 14.218712225556374, 15.199700370430946, 16.18068851530552, 17.1616543084383, 18.14264990389347, 19.12362314760685, 20.104600116610527, 21.085593849420547, 22.06655777990818, 23.04754965007305, 24.028532207012177, 25.009501725435257, 25.990497320890427, 26.971468701958656, 27.952449396252632, 28.933441266417503, 29.914407059550285, 30.895398929715157, 31.876377761363983, 32.85735100507736, 33.83834660053253, 34.819312393665314, 35.80030053853989, 36.78128868341446, 37.76225447654724, 38.74325007200241, 39.72422331571579, 40.70520028471947, 41.68619588017464, 42.66716167330742, 43.64814981818199, 44.62913051247597, 45.610103756189346, 46.591099351644516, 47.572068870067596, 48.55283349752426, 49.45637285709381)
(0.310971736907959, 615.5974731445312, 132632.40625, 6561159.5, 118574152.0, 1055504768.0, 5644763136.0, 20762695680.0, 57720373248.0, 129665622016.0, 2472787

Beam filter validation is not implemented
(12.256750836968422, 13.237746432423592, 14.218712225556374, 15.199700370430946, 16.18068851530552, 17.1616543084383, 18.14264990389347, 19.12362314760685, 20.104600116610527, 21.085593849420547, 22.06655777990818, 23.04754965007305, 24.028532207012177, 25.009501725435257, 25.990497320890427, 26.971468701958656, 27.952449396252632, 28.933441266417503, 29.914407059550285, 30.895398929715157, 31.876377761363983, 32.85735100507736, 33.83834660053253, 34.819312393665314, 35.80030053853989, 36.78128868341446, 37.76225447654724, 38.74325007200241, 39.72422331571579, 40.70520028471947, 41.68619588017464, 42.66716167330742, 43.64814981818199, 44.62913051247597, 45.610103756189346, 46.591099351644516, 47.572068870067596, 48.55283349752426, 49.45637285709381)
(0.0004780908639077097, 1.219215750694275, 416.73748779296875, 34701.03125, 1046490.8125, 14972427.0, 122699328.0, 658383808.0, 2550946048.0, 7663816192.0, 18842478592.0, 39485583360.0, 72734515200.

Beam filter validation is not implemented
(12.256750836968422, 13.237746432423592, 14.218712225556374, 15.199700370430946, 16.18068851530552, 17.1616543084383, 18.14264990389347, 19.12362314760685, 20.104600116610527, 21.085593849420547, 22.06655777990818, 23.04754965007305, 24.028532207012177, 25.009501725435257, 25.990497320890427, 26.971468701958656, 27.952449396252632, 28.933441266417503, 29.914407059550285, 30.895398929715157, 31.876377761363983, 32.85735100507736, 33.83834660053253, 34.819312393665314, 35.80030053853989, 36.78128868341446, 37.76225447654724, 38.74325007200241, 39.72422331571579, 40.70520028471947, 41.68619588017464, 42.66716167330742, 43.64814981818199, 44.62913051247597, 45.610103756189346, 46.591099351644516, 47.572068870067596, 48.55283349752426, 49.45637285709381)
(0.0004780908639077097, 1.219215750694275, 416.73748779296875, 34701.03125, 1046490.8125, 14972427.0, 122699328.0, 658383808.0, 2550946048.0, 7663816192.0, 18842478592.0, 39485583360.0, 72734515200.

Beam filter validation is not implemented
(12.256750836968422, 13.237746432423592, 14.218712225556374, 15.199700370430946, 16.18068851530552, 17.1616543084383, 18.14264990389347, 19.12362314760685, 20.104600116610527, 21.085593849420547, 22.06655777990818, 23.04754965007305, 24.028532207012177, 25.009501725435257, 25.990497320890427, 26.971468701958656, 27.952449396252632, 28.933441266417503, 29.914407059550285, 30.895398929715157, 31.876377761363983, 32.85735100507736, 33.83834660053253, 34.819312393665314, 35.80030053853989, 36.78128868341446, 37.76225447654724, 38.74325007200241, 39.72422331571579, 40.70520028471947, 41.68619588017464, 42.66716167330742, 43.64814981818199, 44.62913051247597, 45.610103756189346, 46.591099351644516, 47.572068870067596, 48.55283349752426, 49.45637285709381)
(0.0004780908639077097, 1.219215750694275, 416.73748779296875, 34701.03125, 1046490.8125, 14972427.0, 122699328.0, 658383808.0, 2550946048.0, 7663816192.0, 18842478592.0, 39485583360.0, 72734515200.

Beam filter validation is not implemented
(12.256750836968422, 13.237746432423592, 14.218712225556374, 15.199700370430946, 16.18068851530552, 17.1616543084383, 18.14264990389347, 19.12362314760685, 20.104600116610527, 21.085593849420547, 22.06655777990818, 23.04754965007305, 24.028532207012177, 25.009501725435257, 25.990497320890427, 26.971468701958656, 27.952449396252632, 28.933441266417503, 29.914407059550285, 30.895398929715157, 31.876377761363983, 32.85735100507736, 33.83834660053253, 34.819312393665314, 35.80030053853989, 36.78128868341446, 37.76225447654724, 38.74325007200241, 39.72422331571579, 40.70520028471947, 41.68619588017464, 42.66716167330742, 43.64814981818199, 44.62913051247597, 45.610103756189346, 46.591099351644516, 47.572068870067596, 48.55283349752426, 49.45637285709381)
(0.0004780908639077097, 1.219215750694275, 416.73748779296875, 34701.03125, 1046490.8125, 14972427.0, 122699328.0, 658383808.0, 2550946048.0, 7663816192.0, 18842478592.0, 39485583360.0, 72734515200.

In [12]:
gvxr.getSourcePosition("m")
gvxr.getDetectorPosition("m")
gvxr.getPhotonCountsPerCm2At1m()

()

In [13]:
print(diad_ui.filter_material_type_widget)

Select(layout=Layout(width='max-content'), options=('None',), value='None')


In [14]:
raise IOError("TOTO")

OSError: TOTO

Instantiate the twin

In [ ]:
twin = None

Display the UI

In [ ]:
creatreUI(twin)

## Getting the data ready

Where to save the data.

In [ ]:
output_path = "../notebooks/output_data/DIAD"
if not os.path.exists(output_path):
    os.makedirs(output_path)

## 1. Set the simulation environment

In [ ]:
# Create an OpenGL context
print("Create an OpenGL context");
gvxr.createOpenGLContext();

In [ ]:
rotation_centre = [0, 0, 0]

In [ ]:
# Create a source
print("Set up the beam")
gvxr.setSourcePosition(-40.0,  0.0, 0.0, "cm")
gvxr.usePointSource()
#  For a parallel source, use gvxr.useParallelBeam()

In [ ]:
# Set its spectrum, here a monochromatic beam
# 1000 photons of 80 keV (i.e. 0.08 MeV) per ray
gvxr.addEnergyBinToSpectrumPerPixelAtSDD(33, "keV", 97)
gvxr.addEnergyBinToSpectrumPerPixelAtSDD(66, "keV", 2)
gvxr.addEnergyBinToSpectrumPerPixelAtSDD(99, "keV", 1)

In [ ]:
# Set up the detector
print("Set up the detector")
gvxr.setDetectorPosition(10.0, 0.0, 0.0, "cm")
gvxr.setDetectorUpVector(0, 0, 1)
gvxr.setDetectorNumberOfPixels(1000, 3)
gvxr.setDetectorPixelSize(1.0, 1.0, "um")
gvxr.setScintillator("GGG", 170, "um")

## 2. Create the sample from a segmented image

Read the image

In [ ]:
labels = imread("../data/labels.tif")
voxel_size = [1.5, 1.5, 1.5]
unit = "um"

Display the image

In [ ]:
plt.figure()
plt.imshow(labels,
    origin='upper',
    extent=[0.0, voxel_size[0] * labels.shape[1],
        0.0, voxel_size[1] * labels.shape[0]]
)
plt.xlabel("Pixel position [" + unit + "]")
plt.ylabel("Pixel position [" + unit + "]")
plt.show()

Make sure it is a 3D image

In [ ]:
if len(labels.shape) == 2:
    labels.shape = [1, *labels.shape]

Associate the labels with actual materials

In [ ]:
material_composition = {
    112: {
        'material type': 'element', 
        'material': 'C', 
        'density': 2.26
    }
    ,

    170: {
        'material type': 'compound', 
        'material': 'SiC', 
        'density': 3.21
    },

    202: {
        'material type': 'mixture', 
        'material': 'Ti90Al6V4', 
        'density': 4.43
    }
}

Process the segmentation to create iso surfaces

In [ ]:
for label in tqdm(np.unique(labels)):

    if label in material_composition.keys():
        selected_material = material_composition[label]
        mesh_label = material_composition[label]['material']

        print("Process", mesh_label)

        # Select the structure
        binary_image = (labels == label).astype(np.uint8)

        # Apply the Marching cubes
        gvxr.makeIsoSurface(mesh_label,
            binary_image,
            1,
            *rotation_centre,
            *voxel_size,
            "um"
        )

        # Save the mesh as an STL file
        gvxr.saveSTLfile(mesh_label, os.path.join(output_path, mesh_label+".stl"))

        # Set the material
        if selected_material['material type'].upper() == 'ELEMENT':
            print("\tUse element", mesh_label)
            gvxr.setElement(mesh_label, mesh_label)

            if "density" in selected_material:
                gvxr.setDensity(mesh_label, selected_material["density"], "g/cm3")

        elif selected_material['material type'].upper() == 'COMPOUND':
            print("\tUse compound", mesh_label)
            gvxr.setCompound(mesh_label, mesh_label)
            gvxr.setDensity(mesh_label, selected_material["density"], "g/cm3")
        
        elif selected_material['material type'].upper() == 'MIXTURE':
            print("\tUse mixture", mesh_label)
            gvxr.setMixture(mesh_label, mesh_label)
            gvxr.setDensity(mesh_label, selected_material["density"], "g/cm3")
        
        else:
            raise IOError("Invalid material type")

        # Add the material
        gvxr.addPolygonMeshAsInnerSurface(mesh_label)
    else:
        print(label, "is not in", material_composition)


Simulate the CT scan

In [ ]:
rotation_centre = [0, 0, 0]
number_of_projections = 3000

gvxr.computeCTAcquisition(
    os.path.join(output_path, "projections"), # The path where the X-ray projections will be saved
    "", # The path where the screenshots will be saved
    number_of_projections, # The total number of projections to simulate
    0, # The rotation angle corresponding to the first projection
    False, # A boolean flag to include or exclude the last angle
    360, # The rotation angle corresponding to the first projection
    50, # The number of white images used to perform the flat-field correction
    *rotation_centre, # The location of the rotation centre
    "mm", # The corresponding unit of length
    0, 0, 1 # The rotation axis
)  

Read the simulated data with CIL

In [ ]:
reader = gVXRDataReader(gvxr.getProjectionOutputPathCT(),
    gvxr.getAngleSetCT(),
    rotation_centre)

data = reader.read()

In [ ]:
print("data.geometry", data.geometry)

Apply the minus log transformation (use use white_level=1.0 as the flat-field correction is already applied)

In [ ]:
data_corr = TransmissionAbsorptionConverter(white_level=1.0)(data)

In [ ]:
data_corr.reorder(order='tigre')

We only want to reconstruct the slice in the middle of the volume

In [ ]:
ig = data_corr.geometry.get_ImageGeometry();

ig.voxel_num_z = 1
print("Image geometry", ig)

In [ ]:
# Perform the reconstruction with CIL
FDK_reconstruction = FDK(data_corr, ig).run()

Apply a circular mask as we simulated a region of interest scan

In [ ]:
FDK_reconstruction.apply_circular_mask()

Save the reconstructed CT images

In [ ]:
writer = TIFFWriter(data=FDK_reconstruction, file_name=os.path.join(output_path, "recons-" + str(number_of_projections), "slice_"), compression="uint16");
writer.write();

Show the CT slice

In [ ]:
show2D(FDK_reconstruction)

# Cleaning up

Once we have finished, it is good practice to clean up the OpenGL contexts and windows with the following command. Note that due to the object-oriented programming nature of the core API of gVXR, this step is automatic anyway.

In [ ]:
gvxr.destroy()